Metodo Voraz

In [ ]:
import pandas as pd
import numpy as np

# Cargar el archivo CSV localmente
file_path = '/content/matrizfinal.csv'  # Ajusta el nombre del archivo si es necesario

try:
    # Cargar la matriz de distancias desde el archivo CSV
    matrix_data = pd.read_csv(file_path)  # Cambia esta línea si el delimitador no es la coma
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta especificada: {file_path}")
    raise
except ValueError as e:
    print(f"Error al leer el archivo CSV: {e}")
    raise

# Procesar la matriz: convertirla a un formato numérico
matrix = matrix_data.iloc[:, 1:].to_numpy()  # La matriz numérica empieza desde la segunda columna
nombres = matrix_data.iloc[:, 0].tolist()  # La primera columna contiene los nombres

# Algoritmo voraz para el TSP
def tsp_voraz(matriz, inicio=0):
    n = len(matriz)
    visitados = [False] * n
    ruta = [inicio]
    visitados[inicio] = True

    for _ in range(n - 1):
        ultimo = ruta[-1]
        # Buscar el vecino no visitado más cercano
        min_dist = float('inf')
        siguiente = -1
        for i in range(n):
            if not visitados[i] and 0 <= matriz[ultimo][i] < min_dist:
                min_dist = matriz[ultimo][i]
                siguiente = i
        if siguiente == -1:
            print("Error: No se encontraron nodos disponibles para continuar la ruta.")
            return None
        ruta.append(siguiente)
        visitados[siguiente] = True

    # Regresar al punto inicial
    ruta.append(inicio)
    return ruta

# Calcular la distancia total de la ruta
def calcular_distancia_total(matriz, ruta):
    distancia = 0
    for i in range(len(ruta) - 1):
        distancia += matriz[ruta[i]][ruta[i+1]]
    return distancia

# Función para generar múltiples rutas
def generar_rutas_diferentes(matriz, n_rutas=4):
    rutas = []
    for i in range(n_rutas):
        inicio = i  # Cambiar el punto de inicio para cada ruta
        ruta_indices = tsp_voraz(matriz, inicio)
        if ruta_indices:
            rutas.append((ruta_indices, calcular_distancia_total(matriz, ruta_indices)))
    return rutas

# Generar las rutas
rutas = generar_rutas_diferentes(matrix, n_rutas=4)

# Mostrar los resultados
for idx, (ruta_indices, distancia_total) in enumerate(rutas):
    ruta_nombres = [nombres[i] for i in ruta_indices]
    print(f"Ruta {idx+1}: {' -> '.join(ruta_nombres)}")
    print(f"Distancia total: {distancia_total}")
    # Mostrar las conexiones de la ruta
    conexiones = [f"{nombres[ruta_indices[i]]} -> {nombres[ruta_indices[i+1]]}" for i in range(len(ruta_indices)-1)]
    print("Conexiones:", " ; ".join(conexiones))
    print("\n")


Ruta 1: Centro Comercial_1 -> Estación_25 -> Museo_15 -> Parque_34 -> Banco_39 -> Parque_35 -> Museo_12 -> Hotel_9 -> Museo_22 -> Estadio_27 -> Parque_26 -> Estación_40 -> Mercado_24 -> Estadio_16 -> Hospital_3 -> Estación_46 -> Estadio_10 -> Mercado_43 -> Hotel_28 -> Hospital_47 -> Escuela_5 -> Parque_19 -> Escuela_29 -> Museo_44 -> Estadio_20 -> Estadio_38 -> Estadio_18 -> Estación_48 -> Hospital_14 -> Hospital_17 -> Escuela_33 -> Estadio_41 -> Banco_31 -> Hotel_36 -> Banco_7 -> Escuela_8 -> Museo_13 -> Escuela_11 -> Banco_45 -> Hotel_42 -> Parque_23 -> Banco_32 -> Escuela_50 -> Estadio_4 -> Museo_6 -> Centro Comercial_2 -> Banco_21 -> Mercado_49 -> Museo_37 -> Estación_30 -> Centro Comercial_1
Distancia total: 3.0928832160000006
Conexiones: Centro Comercial_1 -> Estación_25 ; Estación_25 -> Museo_15 ; Museo_15 -> Parque_34 ; Parque_34 -> Banco_39 ; Banco_39 -> Parque_35 ; Parque_35 -> Museo_12 ; Museo_12 -> Hotel_9 ; Hotel_9 -> Museo_22 ; Museo_22 -> Estadio_27 ; Estadio_27 -> Parqu

Dijkstra

In [ ]:
import pandas as pd
import numpy as np
import heapq

# Cargar el archivo CSV localmente
file_path = '/content/matrizfinal.csv'  # Ajusta el nombre del archivo si es necesario

try:
    # Cargar la matriz de distancias desde el archivo CSV
    matrix_data = pd.read_csv(file_path)  # Cambia esta línea si el delimitador no es la coma
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta especificada: {file_path}")
    raise
except ValueError as e:
    print(f"Error al leer el archivo CSV: {e}")
    raise

# Procesar la matriz: convertirla a un formato numérico
matrix = matrix_data.iloc[:, 1:].to_numpy()  # La matriz numérica empieza desde la segunda columna
nombres = matrix_data.iloc[:, 0].tolist()  # La primera columna contiene los nombres

# Algoritmo de Dijkstra para encontrar el camino más corto entre dos nodos
def dijkstra(matriz, inicio):
    n = len(matriz)
    distancias = [float('inf')] * n
    distancias[inicio] = 0
    pq = [(0, inicio)]  # (distancia, nodo)
    visitados = [False] * n

    while pq:
        dist, nodo_actual = heapq.heappop(pq)
        if visitados[nodo_actual]:
            continue
        visitados[nodo_actual] = True

        for vecino in range(n):
            if not visitados[vecino]:
                nueva_distancia = dist + matriz[nodo_actual][vecino]
                if nueva_distancia < distancias[vecino]:
                    distancias[vecino] = nueva_distancia
                    heapq.heappush(pq, (nueva_distancia, vecino))

    return distancias

# Función para generar todas las distancias más cortas entre todos los nodos usando Dijkstra
def obtener_distancias_completas(matriz):
    distancias_completas = {}
    for i in range(len(matriz)):
        distancias_completas[i] = dijkstra(matriz, i)
    return distancias_completas

# Algoritmo aproximado para el TSP usando distancias calculadas con Dijkstra
def tsp_dijkstra(distancias, inicio=0):
    n = len(distancias)
    visitados = [False] * n
    ruta = [inicio]
    visitados[inicio] = True

    for _ in range(n - 1):
        ultimo = ruta[-1]
        # Buscar el vecino no visitado más cercano usando las distancias calculadas por Dijkstra
        min_dist = float('inf')
        siguiente = -1
        for i in range(n):
            if not visitados[i] and distancias[ultimo][i] < min_dist:
                min_dist = distancias[ultimo][i]
                siguiente = i
        if siguiente == -1:
            print("Error: No se encontraron nodos disponibles para continuar la ruta.")
            return None
        ruta.append(siguiente)
        visitados[siguiente] = True

    # Regresar al punto inicial
    ruta.append(inicio)
    return ruta

# Calcular la distancia total de la ruta
def calcular_distancia_total(matriz, ruta):
    distancia = 0
    for i in range(len(ruta) - 1):
        distancia += matriz[ruta[i]][ruta[i+1]]
    return distancia

# Función para generar múltiples rutas
def generar_rutas_diferentes(matriz, n_rutas=4):
    # Calcular las distancias más cortas entre todos los nodos usando Dijkstra
    distancias_completas = obtener_distancias_completas(matriz)

    rutas = []
    for i in range(n_rutas):
        inicio = i  # Cambiar el punto de inicio para cada ruta
        ruta_indices = tsp_dijkstra(distancias_completas, inicio)
        if ruta_indices:
            rutas.append((ruta_indices, calcular_distancia_total(matriz, ruta_indices)))
    return rutas

# Generar las rutas
rutas = generar_rutas_diferentes(matrix, n_rutas=4)

# Mostrar los resultados
for idx, (ruta_indices, distancia_total) in enumerate(rutas):
    ruta_nombres = [nombres[i] for i in ruta_indices]
    print(f"Ruta {idx+1}: {' -> '.join(ruta_nombres)}")
    print(f"Distancia total: {distancia_total}")
    # Mostrar las conexiones de la ruta
    conexiones = [f"{nombres[ruta_indices[i]]} -> {nombres[ruta_indices[i+1]]}" for i in range(len(ruta_indices)-1)]
    print("Conexiones:", " ; ".join(conexiones))
    print("\n")


Ruta 1: Centro Comercial_1 -> Estación_25 -> Museo_15 -> Parque_34 -> Banco_39 -> Parque_35 -> Museo_12 -> Hotel_9 -> Museo_22 -> Estadio_27 -> Parque_26 -> Estación_40 -> Mercado_24 -> Estadio_16 -> Hospital_3 -> Estación_46 -> Estadio_10 -> Banco_7 -> Escuela_8 -> Museo_13 -> Estadio_4 -> Museo_6 -> Estación_48 -> Hospital_14 -> Estadio_18 -> Hospital_17 -> Escuela_33 -> Estadio_41 -> Banco_31 -> Escuela_5 -> Parque_19 -> Escuela_29 -> Museo_44 -> Estadio_20 -> Hotel_28 -> Banco_45 -> Mercado_43 -> Banco_21 -> Parque_23 -> Banco_32 -> Escuela_50 -> Hotel_36 -> Hotel_42 -> Hospital_47 -> Centro Comercial_2 -> Museo_37 -> Estación_30 -> Mercado_49 -> Escuela_11 -> Estadio_38 -> Centro Comercial_1
Distancia total: 7.079910230999999
Conexiones: Centro Comercial_1 -> Estación_25 ; Estación_25 -> Museo_15 ; Museo_15 -> Parque_34 ; Parque_34 -> Banco_39 ; Banco_39 -> Parque_35 ; Parque_35 -> Museo_12 ; Museo_12 -> Hotel_9 ; Hotel_9 -> Museo_22 ; Museo_22 -> Estadio_27 ; Estadio_27 -> Parque

A*

In [ ]:
import pandas as pd
import numpy as np
import heapq

# Cargar el archivo CSV localmente
file_path = '/content/matrizfinal.csv'  # Ajusta el nombre del archivo si es necesario

try:
    # Cargar la matriz de distancias desde el archivo CSV
    matrix_data = pd.read_csv(file_path)  # Cambia esta línea si el delimitador no es la coma
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta especificada: {file_path}")
    raise
except ValueError as e:
    print(f"Error al leer el archivo CSV: {e}")
    raise

# Procesar la matriz: convertirla a un formato numérico
matrix = matrix_data.iloc[:, 1:].to_numpy()  # La matriz numérica empieza desde la segunda columna
nombres = matrix_data.iloc[:, 0].tolist()  # La primera columna contiene los nombres

# Algoritmo A* para encontrar el camino más corto entre dos nodos
def a_star(matriz, inicio, objetivo, n):
    # Heurística: distancia en línea recta (simplificada como la distancia más corta entre dos nodos)
    def heuristica(a, b):
        return matriz[a][b]

    # Costos
    g_costs = {inicio: 0}  # Costos desde el nodo de inicio
    f_costs = {inicio: heuristica(inicio, objetivo)}  # f = g + h
    came_from = {}  # Para reconstruir la ruta
    open_list = [(f_costs[inicio], inicio)]  # Lista de nodos a explorar (f, nodo)
    closed_list = set()  # Nodos ya explorados

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == objetivo:
            # Reconstruir la ruta desde el objetivo
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(inicio)
            return path[::-1]  # Retorna el camino invertido

        closed_list.add(current)

        for neighbor in range(n):
            if neighbor in closed_list or matriz[current][neighbor] == 0:
                continue

            tentative_g_cost = g_costs[current] + matriz[current][neighbor]

            if neighbor not in g_costs or tentative_g_cost < g_costs[neighbor]:
                came_from[neighbor] = current
                g_costs[neighbor] = tentative_g_cost
                f_costs[neighbor] = tentative_g_cost + heuristica(neighbor, objetivo)
                heapq.heappush(open_list, (f_costs[neighbor], neighbor))

    return None  # Si no se encuentra una ruta

# Algoritmo A* para el TSP
def tsp_a_star(matriz, inicio=0):
    n = len(matriz)
    visitados = [False] * n
    ruta = [inicio]
    visitados[inicio] = True

    for _ in range(n - 1):
        ultimo = ruta[-1]
        # Buscar el nodo no visitado más cercano utilizando A*
        min_cost = float('inf')
        siguiente = -1
        for i in range(n):
            if not visitados[i]:
                path = a_star(matriz, ultimo, i, n)
                if path is not None:
                    costo_total = sum(matriz[path[j]][path[j+1]] for j in range(len(path)-1))
                    if costo_total < min_cost:
                        min_cost = costo_total
                        siguiente = i
        if siguiente == -1:
            print("Error: No se encontraron nodos disponibles para continuar la ruta.")
            return None
        ruta.append(siguiente)
        visitados[siguiente] = True

    # Regresar al punto inicial
    ruta.append(inicio)
    return ruta

# Calcular la distancia total de la ruta
def calcular_distancia_total(matriz, ruta):
    distancia = 0
    for i in range(len(ruta) - 1):
        distancia += matriz[ruta[i]][ruta[i+1]]
    return distancia

# Función para generar múltiples rutas
def generar_rutas_diferentes(matriz, n_rutas=4):
    rutas = []
    for i in range(n_rutas):
        inicio = i  # Cambiar el punto de inicio para cada ruta
        ruta_indices = tsp_a_star(matriz, inicio)
        if ruta_indices:
            rutas.append((ruta_indices, calcular_distancia_total(matriz, ruta_indices)))
    return rutas

# Generar las rutas
rutas = generar_rutas_diferentes(matrix, n_rutas=4)

# Mostrar los resultados
for idx, (ruta_indices, distancia_total) in enumerate(rutas):
    ruta_nombres = [nombres[i] for i in ruta_indices]
    print(f"Ruta {idx+1}: {' -> '.join(ruta_nombres)}")
    print(f"Distancia total: {distancia_total}")
    # Mostrar las conexiones de la ruta
    conexiones = [f"{nombres[ruta_indices[i]]} -> {nombres[ruta_indices[i+1]]}" for i in range(len(ruta_indices)-1)]
    print("Conexiones:", " ; ".join(conexiones))
    print("\n")


Ruta 1: Centro Comercial_1 -> Estación_25 -> Museo_15 -> Parque_34 -> Banco_39 -> Parque_35 -> Museo_12 -> Hotel_9 -> Museo_22 -> Estadio_27 -> Parque_26 -> Estación_40 -> Mercado_24 -> Estadio_16 -> Hospital_3 -> Estación_46 -> Estadio_10 -> Banco_7 -> Escuela_8 -> Museo_13 -> Estadio_4 -> Museo_6 -> Estación_48 -> Hospital_14 -> Estadio_18 -> Hospital_17 -> Escuela_33 -> Estadio_41 -> Banco_31 -> Escuela_5 -> Parque_19 -> Escuela_29 -> Museo_44 -> Estadio_20 -> Museo_37 -> Escuela_50 -> Hotel_36 -> Centro Comercial_2 -> Banco_45 -> Mercado_43 -> Hotel_28 -> Hotel_42 -> Hospital_47 -> Estadio_38 -> Parque_23 -> Banco_32 -> Banco_21 -> Escuela_11 -> Estación_30 -> Mercado_49 -> Centro Comercial_1
Distancia total: 6.155149281999998
Conexiones: Centro Comercial_1 -> Estación_25 ; Estación_25 -> Museo_15 ; Museo_15 -> Parque_34 ; Parque_34 -> Banco_39 ; Banco_39 -> Parque_35 ; Parque_35 -> Museo_12 ; Museo_12 -> Hotel_9 ; Hotel_9 -> Museo_22 ; Museo_22 -> Estadio_27 ; Estadio_27 -> Parque